# Practical Course 1

In [1]:
import sys
sys.path.append('../../rai/rai/ry')
import libry as ry
import numpy as np

In [2]:
class RaiRobot():
    
    def __init__(self, nodeName:str, fileName:str):
        self.C = ry.Config()
        self.D = self.C.view()
        self.C.clear()
        self.C.addFile(fileName)
        self.q_home = self.C.getJointState()
        self.q_zero = self.q_home * 0.
        self.B = self.C.operate(nodeName)
        self.B.sync(self.C)
        self.real = False
        self.B.sendToReal(self.real)
        self.C.addObject(name="ball", shape=ry.ST.sphere, size=[.1], pos=[.8,.8,1.5], color=[1,1,0])
    
    def getFrameNames(self)-> list:
        return self.C.getFrameNames()
    
    def moveToPosition(self, pos:np.ndarray, frameName:str):
        sigma = 1e-2
        F = self.C.feature(ry.FS.position, [frameName])
        self.inverseKinematics([pos], [F], sigma, 1000, 0.01)
    
    def gazeAt(self, frameName, goalFrameName):
        sigma = 1e-2
        F = self.C.feature(ry.FS.gazeAt, [frameName, goalFrameName])
        self.inverseKinematics([0, 0], F, sigma, 1000, 0.01)
        
    def inverseKinematics(self, task, features, sigma, steps, stepsize):
        q = self.C.getJointState()
        n = self.C.getJointDimension()
        W = sigma * np.eye(n)
        y, J = self._evalFeatures(features)
        for i in range(steps):
            q += stepsize * np.linalg.inv(J.T @ J + W) @ J.T @ (task - y)
            self.C.setJointState(q)
            self.B.move([q], [10.0], False)
            y, J = self._evalFeatures(features)
    
    
    def _evalFeatures(self, features):
        y_temp = list(map(lambda f: f.eval(self.C), features))
        print(y_temp)
        y, J = tuple(map(lambda x: np.vstack(list(x)), zip(*y_temp)))
        return np.squeeze(y), np.squeeze(J)
    
    
    def goHome(self):
        self.B.move([self.q_home], [10.0], False)
    
    def sendToReal(self, val:bool):
        self.real = val
        self.B.sendToReal(val)
    

In [3]:
def reset(robot):
    robot.C = 0
    robot.D = 0
    robot.B = 0
    return RaiRobot('', '../../rai-robotModels/baxter/baxter.g')

In [4]:
robot =  RaiRobot('', '../../rai-robotModels/baxter/baxter.g')

In [10]:
robot = reset(robot)

In [11]:
robot.moveToPosition([.8, .8, 1.5], 'baxterL')

[(array([0.58181738, 0.19057273, 1.12082012]), array([[ 0.00000000e+00,  0.00000000e+00,  6.84542579e-02,
         0.00000000e+00, -2.12535776e-01,  0.00000000e+00,
        -2.54224773e-03,  0.00000000e+00, -5.59705733e-01,
         0.00000000e+00,  2.28657101e-01,  0.00000000e+00,
        -2.70622416e-01,  0.00000000e+00,  4.59701721e-17,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  5.17790057e-01,
         0.00000000e+00, -1.80987858e-01,  0.00000000e+00,
         5.06333753e-01,  0.00000000e+00,  6.46664085e-02,
         0.00000000e+00,  2.37879597e-01,  0.00000000e+00,
         2.60814624e-01,  0.00000000e+00,  1.50920942e-16,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -1.59139758e-08,
         0.00000000e+00, -2.80838774e-01,  0.00000000e+00,
        -2.09540848e-01,  0.00000000e+00, -2.75166778e-01,
         0.00000000e+00,  1.65687755e-02,  0.00000000e+00,
        -9.82575642e-03,  0.00000000e+00,

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 3)

In [8]:
robot.gazeAt('baxterL', 'ball')

In [10]:
F = robot.C.feature(ry.FS.position, ['baxterL'])
F.eval(robot.C)

(array([0.79996444, 0.79995431, 1.49998545]),
 array([[ 0.00000000e+00,  0.00000000e+00, -5.40927181e-01,
          0.00000000e+00,  4.74154699e-02,  0.00000000e+00,
         -4.05030948e-01,  0.00000000e+00, -9.58926750e-02,
          0.00000000e+00, -4.57530252e-02,  0.00000000e+00,
         -3.06921971e-01,  0.00000000e+00,  1.38777878e-17,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  7.35936984e-01,
          0.00000000e+00,  8.80550588e-02,  0.00000000e+00,
          6.25745550e-01,  0.00000000e+00, -2.25001246e-01,
          0.00000000e+00,  1.95998477e-01,  0.00000000e+00,
          1.08974451e-01,  0.00000000e+00,  2.77555756e-17,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.25752616e-07,
          0.00000000e+00, -7.56183274e-01,  0.00000000e+00,
         -1.56386687e-01,  0.00000000e+00, -6.06819942e-01,
          0.00000000e+00,  1.88474151e-01,  0.00000000e+00,
         -1.87831974e-01, 

In [10]:
def moveToPosition(pos):
    q = np.zeros(C.getJointDimension())
    C.setJointState(q)
    sigma = 1e-2
    W = sigma * np.eye(C.getJointDimension())
    F = C.feature(FS.position, ['baxterR'])
    y, J = F.eval(C)
    stepsize = 0.1
    for i in range(10):
        
        q += stepsize * np.linalg.inv(J.T @ J + W) @ J.T @ (pos - y)
        C.setJointState(q)
        B.move([q], [10.0], False)
        y, J = F.eval(C)

    